In [3]:
import warnings
warnings.simplefilter("ignore", UserWarning)

from __future__ import print_function
from pytorch_lightning.metrics.functional import accuracy

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import pickle
import argparse
from argparse import RawTextHelpFormatter
import fnmatch
import os
import json
import random
import numpy as np
import shutil
import traceback
import copy

import os
from os import listdir
from os.path import isfile, join

import numpy as np

from tqdm import tqdm
import random 
from matplotlib import pyplot as plt

import pandas as pd

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms


import torch
import pytorch_lightning as pl
import os

from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint

from torch.nn import functional as F
import torchmetrics

from pytorch_lightning.loggers import TensorBoardLogger
from pl_bolts.callbacks import PrintTableMetricsCallback


import math
import torchvision.models as models
from PIL import Image

from skimage import io as iio

In [2]:
# 執行一次即可，用來下載參數
torch.hub.list('zhanghang1989/ResNeSt', force_reload=True)

Downloading: "https://github.com/zhanghang1989/ResNeSt/archive/master.zip" to C:\Users\bbb50/.cache\torch\hub\master.zip


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [4]:
class pre_resnest50(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size



        backbone = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        
        self.feature_extractor = nn.Sequential(*layers)

        num_target_classes = 800
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)


        return F.log_softmax(x, dim=1)

    def configure_optimizers(self):
        
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        lr_scheduler = {'scheduler': torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95),
                        'name': 'expo_lr'}
        return [optimizer], [lr_scheduler]
    
    def training_step(self,batch,batch_idx):
        
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        
        self.log('train_loss',loss, prog_bar=True , on_step=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True , on_step=False, on_epoch=True)
        return loss


    def validation_step(self,batch,batch_idx):
        
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        
        self.log('val_loss', loss, prog_bar=True , on_step=False, on_epoch=True)
        self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss, acc
    


In [5]:
class pre_resnest50(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size



        backbone = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        
        self.feature_extractor = nn.Sequential(*layers)

        num_target_classes = 800
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)


        return F.log_softmax(x, dim=1)

    def configure_optimizers(self):
        
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        lr_scheduler = {'scheduler': torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95),
                        'name': 'expo_lr'}
        return [optimizer], [lr_scheduler]
    
    def training_step(self,batch,batch_idx):
        
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        
        self.log('train_loss',loss, prog_bar=True , on_step=True, on_epoch=True)
        self.log('train_acc', acc, prog_bar=True , on_step=False, on_epoch=True)
        return loss


    def validation_step(self,batch,batch_idx):
        
        x, y = batch
        logits = self.forward(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)
        
        self.log('val_loss', loss, prog_bar=True , on_step=False, on_epoch=True)
        self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True)
        return loss, acc
    
class pre_resnest50_wo_freeze(pl.LightningModule):
    def __init__(self, learning_rate, batch_size):
        
        super(pre_resnest50_wo_freeze, self).__init__()
        
        self.lr = learning_rate
        self.batch_size = batch_size
        self.save_hyperparameters()

        resnest50 = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        trained_resnest50 = pre_resnest50.load_from_checkpoint('pre_resnest50_epoch=54-val_loss=6.26.ckpt', learning_rate=0.08, batch_size=128)

        layers_1 = list(resnest50.children())[:-1][0]
        layers_2 = list(resnest50.children())[:-1][1:-2]
        layers_3 = list(resnest50.children())[:-1][-2:]
        self.classifier = list(trained_resnest50.children())[-1]

        self.seq_1 = nn.Sequential(*layers_1)
        self.seq_2 = nn.Sequential(*layers_2)
        self.seq_3 = nn.Sequential(*layers_3)



    def forward(self, x):
        
        x = self.seq_1(x)
        
        self.seq_2.eval()
        with torch.no_grad():
            x = self.seq_2(x)
            
        x = self.seq_3(x).flatten(1)    
        x = self.classifier(x)


        return F.log_softmax(x, dim=1)

    def configure_optimizers(self):
        
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        lr_scheduler = {'scheduler': torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95),
                        'name': 'expo_lr'}
        return [optimizer], [lr_scheduler]
    


In [6]:
model = pre_resnest50_wo_freeze.load_from_checkpoint('resnest50_tbrain_epoch=49-val_loss=0.056.ckpt')

Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master
Using cache found in C:\Users\bbb50/.cache\torch\hub\zhanghang1989_ResNeSt_master


In [7]:
# # 讀模型參數
# model = pre_resnest50.load_from_checkpoint('pre_resnest50_epoch=54-val_loss=6.26.ckpt', learning_rate=0.08, batch_size=128)
model.eval()

# 讀 模型輸出類別對應的中文
cls_to_idx = np.load('cls_to_idx.npy')

In [12]:
img = Image.open('0_我/你_2.png').convert('RGB')

In [8]:
type(img)

PIL.Image.Image

In [5]:
img_2 = iio.imread('10838.jpg')

In [7]:
img_2

array([[[196, 147, 130],
        [255, 184, 181],
        [201,  85,  96],
        ...,
        [252, 247, 243],
        [246, 240, 240],
        [242, 238, 239]],

       [[198, 149, 132],
        [255, 185, 182],
        [200,  84,  95],
        ...,
        [252, 247, 244],
        [245, 241, 240],
        [242, 238, 239]],

       [[196, 147, 130],
        [255, 184, 181],
        [200,  84,  95],
        ...,
        [251, 247, 244],
        [245, 241, 238],
        [240, 239, 237]],

       ...,

       [[199, 123, 136],
        [244, 170, 185],
        [195, 128, 146],
        ...,
        [233, 230, 225],
        [231, 226, 223],
        [234, 225, 228]],

       [[204, 128, 141],
        [249, 175, 190],
        [199, 132, 150],
        ...,
        [233, 230, 225],
        [231, 226, 223],
        [234, 225, 228]],

       [[206, 130, 143],
        [251, 177, 192],
        [202, 135, 153],
        ...,
        [233, 230, 225],
        [231, 226, 223],
        [234, 225, 228]]

In [8]:


# 讀入照片
img = Image.open('10838.jpg').convert('RGB')
#
data_transform = transforms.Compose([
                            transforms.Resize(size = (224, 224)),
                            transforms.Grayscale(num_output_channels=3),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
img_input = data_transform(img)
img_input = img_input.reshape(1,*img_input.shape)

y_hat = model(img_input)

# output 為單一中文字
return_chinese = cls_to_idx[torch.argmax(y_hat, dim=1)[0]]
print(return_chinese)

梁
